# Classification with SHAP

- This tutorial demonstrates how to use structured binary classification with Keras, starting from a raw CSV file. 


- This example is an advanced version of [](structured_data_classification_intro.ipynb) since we will use more functions and less code.

## Setup

In [5]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers

import shap

import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow: {tf.__version__}")
print(f"SHAP: {shap.__version__}")

TensorFlow: 2.8.1
SHAP: 0.40.0


## Data

- We use the features below to predict whether a patient has a heart disease (`Target`).

featureumn| Description| Feature Type
------------|--------------------|----------------------
Age | Age in years | Numerical
Sex | (1 = male; 0 = female) | Categorical
CP | Chest pain type (0, 1, 2, 3, 4) | Categorical
Trestbpd | Resting blood pressure (in mm Hg on admission) | Numerical
Chol | Serum cholesterol in mg/dl | Numerical
FBS | fasting blood sugar in 120 mg/dl (1 = true; 0 = false) | Categorical
RestECG | Resting electrocardiogram results (0, 1, 2) | Categorical
Thalach | Maximum heart rate achieved | Numerical
Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical
Oldpeak | ST depression induced by exercise relative to rest | Numerical
Slope | Slope of the peak exercise ST segment | Numerical
CA | Number of major vessels (0-3) featureored by fluoroscopy | Both numerical & categorical
Thal | normal; fixed defect; reversible defect | Categorical (string)
Target | Diagnosis of heart disease (1 = true; 0 = false) | Target

## Model

### Model import

- Load the model (see [](structured_data_classification_layers.ipynb)): 

In [2]:
model = tf.keras.models.load_model('my_hd_classifier')

2022-05-26 21:30:51.211932: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


- To get a prediction for a new sample, you can simply call the Keras `Model.predict` method.

- There are just two things you need to do:

  - Wrap scalars into a list so as to have a batch dimension (Models only process batches of data, not single samples).

  - Call `tf.convert_to_tensor` on each feature.

In [3]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

In [4]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [5]:
predictions = model.predict(input_dict)

In [6]:
print(
    "This particular patient had a %.1f percent probability "
    "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
)

This particular patient had a 62.4 percent probability of having a heart disease, as evaluated by our model.


In [7]:
X_train = pd.read_csv("df_train.csv")

In [8]:
explainer = shap.KernelExplainer(model.predict, X_train)


Provided model function fails when applied to the provided data set.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
shap_values = explainer.shap_values(X_test.iloc[0,:])
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test.iloc[0,:])